In [1]:
import geemap
import ee
from geemap import geojson_to_ee, ee_to_geojson


In [2]:
new_york= "data/nyc.geojson"

In [3]:
import geopandas as gpd

In [4]:
gdf= gpd.read_file(new_york)
gdf.head()

,boro_code,boro_name,shape_area,shape_leng,geometry
0,3,Brooklyn,1934142775.63,728147.130565,"MULTIPOLYGON (((-73.86327 40.58388, -73.86381 ..."
1,5,Staten Island,1623618684.31,325910.306061,"MULTIPOLYGON (((-74.05051 40.56642, -74.05047 ..."
2,1,Manhattan,636646082.483,360037.525777,"MULTIPOLYGON (((-74.01093 40.68449, -74.01193 ..."
3,2,Bronx,1187174772.32,463180.578878,"MULTIPOLYGON (((-73.89681 40.79581, -73.89694 ..."
4,4,Queens,3041418003.6,888197.010944,"MULTIPOLYGON (((-73.82645 40.59053, -73.82642 ..."


In [5]:
ee.Initialize()

In [6]:
ee_fc = geojson_to_ee(new_york)

In [7]:
collection = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterDate("2022-01-01", "2022-12-31")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 5))
)


In [8]:
image= collection.median()

In [9]:
vis_params = {
    "bands": ["B4", "B3", "B2"],
    "min": 0,
    "max": 2000,
}

In [10]:
subset = image.clip(ee_fc)

In [11]:
landcover= ee.ImageCollection("ESA/WorldCover/v100").select("Map").first()

In [12]:
subset_landcover = landcover.clip(ee_fc)

In [13]:
left_layer = geemap.ee_tile_layer(subset, vis_params, "Sentinel-2")

In [14]:
right_layer = geemap.ee_tile_layer(subset_landcover, {}, "Land Cover")

In [23]:
m= geemap.Map(center=[40.7128, -74.0060], zoom=10)

In [24]:
m.split_map(left_layer, right_layer)
m

Map(center=[40.7128, -74.006], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [83]:
m.addLayer(subset, vis_params, "RGB")
m.centerObject(subset)
m

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [29]:
vis_params = {
    "bands": ["B8", "B4", "B3"],
    "min": 0,
    "max": 2000,
}
m.addLayer(subset, vis_params, "Near Infrared")

In [13]:
landcover= ee.ImageCollection("ESA/WorldCover/v100").select("Map").first()

In [14]:
subset_landcover = landcover.clip(ee_fc)

In [15]:
m.addLayer(subset_landcover, {}, "NYC Land Cover")
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [72]:
legend_dict= {
    "10 Tree cover" : "#006400",
    "20 Shrubland": "#ffbb22",
    "30 Grassland": "#ffff4c",
    "40 Cropland": "#f096ff",
    "50 Built-up": "#a0000",
    "60 Bare areas": "#b4b4b4",
    "70 Snow and ice": "#f0f0f0",
    "80 Permanent Water bodies": "#0064c8",
    "90 Herbaceous wetland": "#0096a0",
    "95 Mangroves": "#00cf75",
    "100 Moss and lichen": "#fae6a0",
}

In [73]:
from ipyleaflet import LegendControl

In [74]:
legend= LegendControl(legend=legend_dict, name="Legend", position="bottomright")
m.add_control(legend)

In [35]:
m.add_legend(builtin_legend="ESA_WorldCover")

In [40]:
text= "NYC Sentinel-2 Near Infrared Image"

In [41]:
m.add_text(text, fontsize= 25, position="topright")

In [42]:
text1= "Made by Zhanchao Yang"
m.add_text(text1, fontsize= 10, position="bottomright")

In [84]:
m.save("nyc_sentinel_2_ imagery.html", title="NYC Sentinel-2 Imagery")

In [85]:
region= m.user_roi
region

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -74.328003,
              40.390489
            ],
            [
              -74.328003,
              40.988192
            ],
            [
              -73.605652,
              40.988192
            ],
            [
              -73.605652,
              40.390489
            ],
            [
              -74.328003,
              40.390489
            ]
          ]
        ]
      },
      "geodesic": {
        "constantValue": false
      }
    }
  }
})

In [88]:
geemap.ee_export_image_to_drive(
    image,
    description="nyc_sentinel_2_imagery",
    folder="export",
    region=region,
    scale=10,
)